In [ ]:
import sys
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.stats import bayes_mvs as bayesest
import time
from tqdm import tqdm

from PyEcoLib.simulator import Simulator
import matplotlib.gridspec as gridspec
import matplotlib

In [ ]:
mean_size = 1  # femto liter
doubling_time = 1  # min
tmax: int = 6  # min
sample_time = 0.05  # min
div_steps = 20
ncells: int = 1000
gr=np.log(2)
sample_time=0.05


In [ ]:
def setaxis(ax):
    for axis in ['bottom','left']:
        ax.spines[axis].set_linewidth(1.5)
    for axis in ['top','right']:
        ax.spines[axis].set_linewidth(0)
    ax.grid()

In [ ]:
gr = np.log(2)/doubling_time

if not os.path.exists('./data'):
    os.makedirs('./data')  # data path
if not os.path.exists('./figures'):
    os.makedirs('./figures')  # Figures path


SINGLE CELL SIMULATION

In [ ]:
#Simulating some cell size trajectories
from PyEcoLib.simulator import Simulator
sim = Simulator(ncells=ncells, gr=gr, sb=mean_size, steps=div_steps,V0array=np.random.gamma(shape=1/0.02,scale=0.02,size=1000)) 
sim.szdyn(tmax=5, sample_time=0.01, nameCRM='./data/SingleLin.csv')

In [ ]:
#function to bin the data for division strategy
def getbins(x,y,qnt):
    
    df=pd.DataFrame({'X':x,'Y':y})
    quantnumber=qnt

    CV2d=[]
    delt=[]
    sb=[]

    errcv2d=[]
    errdelt=[]
    errsb=[]
    bins=np.linspace(np.min(x),np.max(x),qnt+1)
    for i in range(len(bins)-1):
        quanta1=df[df.X>bins[i]]
        quanta2=quanta1[quanta1.X<bins[i+1]] 
        mean_cntr, var_cntr, std_cntr = bayesest(quanta2.Y,alpha=0.95)
        meanv0_cntr, varv0_cntr, stdv0_cntr = bayesest(quanta2.X,alpha=0.95)
        CV2d.append(var_cntr[0]/mean_cntr[0]**2)
        delt.append(mean_cntr[0])
        sb.append(meanv0_cntr[0])
        errv=(var_cntr[1][1]-var_cntr[0])/mean_cntr[0]**2+2*(mean_cntr[1][1]-mean_cntr[0])*var_cntr[0]/mean_cntr[0]**3
        errcv2d.append(errv)
        errdelt.append(mean_cntr[1][1]-mean_cntr[0])
        errsb.append(meanv0_cntr[1][1]-meanv0_cntr[0])
    return (sb,errsb,delt,errdelt)



In [ ]:
sim.divstrat(tmax=10, nameDSM = "./data/dataDSM.csv")

In [ ]:
#Ploting Figure1
fig= plt.figure(figsize=(15,2.5))
gs0 = gridspec.GridSpec(1, 3, figure=fig,wspace=0.2,hspace=0.2)


ax1 = fig.add_subplot(gs0[1])
data=pd.read_csv('./data/SingleLin.csv')
for c in data.columns[10:19]:
    df=data[c]
    ax1.plot(np.linspace(0,5,len(df)),df,c='#bebebe')

df=data[data.columns[7]]
ax1.plot(np.linspace(0,5,len(df)),df,lw=2)
ax1.set_ylim(0,3)
ax1.set_xlim(0,5)
gs00 = gridspec.GridSpecFromSubplotSpec(1,2, subplot_spec=gs0[2],hspace=0.1)
ax20 = fig.add_subplot(gs00[0])

df=pd.read_csv("./data/dataDSM.csv")
ax20.set_ylim(0.2,2.2)
ax20.set_xlim(0.2,2.2)
added=np.random.gamma(shape=20,scale=1/20,size=1000)
ax20.scatter(df.S_b.tolist()[:1000],added,s=1,c='#0192EA')

sb,errsb,delt,errdelt=getbins(df.S_b.tolist()[:1000],added,qnt=5)

plt.plot([np.min(df.S_b),np.max(df.S_b)],[1,1],c='#02447E')

ax20.errorbar(np.array(sb),np.array(delt),xerr=errsb,yerr=errdelt, fmt='o',
                 mec='#02447E',capsize=5,markersize='5',elinewidth=3,c='#0073B9')

ax20.set_ylabel('Added Size')
ax20.set_xlabel('Size at Birth')
setaxis(ax20)
setaxis(ax1)
ax1.set_xlabel('Time')
ax1.set_ylabel('Cell Size')

data=pd.read_csv('./data/SingleLin.csv')
mnarr=[]
for t in data.time.unique():
    datatime=data[data.time==t]
    dataszs=datatime[datatime.columns[1:]]
    mnarr.append(np.mean(dataszs.iloc[0].tolist()))
ax1.plot(data.time.unique(),mnarr,c="#696969")

plt.savefig('./SizeDyn.eps',bbox_inches='tight')

POPULATION SIMULATION

In [ ]:
#population simulator
from PyEcoLib.PopSimulator import PopSimulator
sim = PopSimulator(ncells=500, gr=np.log(2), sb=1, steps=10, nu=2,
                   CV2gr=0.002,CV2div=0.001,V0array=np.random.gamma(shape=1/0.03,scale=0.03,size=500)) 


In [ ]:
sim.szdyn(tmax=4, sample_time=0.01, FileName='./data/dynamics.csv', DivEventsFile='./data/divevents.csv')
pd.read_csv('./data/dynamics.csv')

In [ ]:
#ploting Figure3
fig= plt.figure(figsize=(17,2.5))
gs0 = gridspec.GridSpec(1, 3, figure=fig,wspace=0.2,hspace=0.2)
gs00 = gridspec.GridSpecFromSubplotSpec(1,2, subplot_spec=gs0[0],hspace=0.1)
ax10 = fig.add_subplot(gs00[1])
ax0 = fig.add_subplot(gs0[1])
data=pd.read_csv('./data/dynamics.csv')
data1=data[data.Sample==105]
for cell in data1.Cell.unique():
    datacell=data1[data1.Cell==cell]
    ax0.plot(datacell.Time,datacell.Size)    

gs00 = gridspec.GridSpecFromSubplotSpec(1,2, subplot_spec=gs0[2],hspace=0.1)
ax20 = fig.add_subplot(gs00[0])

data=pd.read_csv('./data/dynamics.csv')

for sample in data.Sample.unique()[1:20]:
    data1=data[data.Sample==sample]
    Narr=[]
    for time in data1.Time.unique():
 
        datatime=data1[data1.Time==time]
        Narr.append(len(datatime))
    ax20.plot(data1.Time.unique(),Narr,c='#bebebe')
    
data1=data[data.Sample==105]
Narr=[]
for time in data1.Time.unique():

    datatime=data1[data1.Time==time]
    Narr.append(len(datatime))
ax20.plot(data1.Time.unique(),Narr)
    
ax20.set_yscale('log')

Narr=[]
for time in data.Time.unique():
    
    datatime=data[data.Time==time]
    Nsample=[]
    for sample in datatime.Sample.unique():
        datas=datatime[datatime.Sample==sample]
        Nsample.append(len(datas))
        
    Narr.append(np.mean(Nsample))
ax20.plot(data1.Time.unique(),smoot(Narr,5),c='#696969')

Narr=[]
for time in data.Time.unique():
    
    datatime=data[data.Time==time]

        
    Narr.append(np.mean(datatime.Size))
ax0.plot(smoot(data1.Time.unique(),3),Narr,c='#696969')

ax20.set_ylabel('Cell Population')
ax20.set_xlabel('Time')
setaxis(ax20)
setaxis(ax0)
ax0.set_xlabel('Time')
ax0.set_xlim(0,4)
ax0.set_ylim(0,2.5)
ax1.set_xlim(0,3)
ax0.set_ylabel('Cell Size')
plt.savefig('./PopDyn.eps',bbox_inches='tight')

In [ ]:
CV2sz = 0.02
tmax=6
sim = Simulator(ncells=1, gr=gr, sb=mean_size, steps=div_steps,lamb=0.5)


In [ ]:
#Numerical solution for the cell size moments dynamics (adder, sizer and timer)
sim = Simulator(ncells=1, gr=gr, sb=mean_size, steps=div_steps,lamb=1)
sim.szdynFSP(tmax=tmax,sample_time=0.05, nameFSP="./data/AdderDyn.csv", CV2sz=CV2sz)

sim = Simulator(ncells=1, gr=gr, sb=mean_size, steps=div_steps,lamb=1)
sim.szdynFSP(tmax=tmax,sample_time=0.05, nameFSP="./data/dataFSPsz.csv", CV2sz=CV2sz)

sim = Simulator(ncells=1, gr=gr, sb=mean_size, steps=div_steps,lamb=0.5)
sim.szdynFSP(tmax=tmax,sample_time=0.05, nameFSP="./data/dataFSPtim.csv", CV2sz=CV2sz)

In [ ]:
#Simulation of the division strategy (adder, sizer and timer)

tmax=10
ncells=5000
start = time.time()
sim = Simulator(ncells=ncells, gr=gr, sb=mean_size, steps=div_steps)
sim.divstrat(tmax=tmax,  nameDSM="./data/dataDSMadder.csv")
print('It took', np.int(time.time()-start), 'seconds.')

ncells=5000
start = time.time()
sim = Simulator(ncells=ncells, gr=gr, sb=mean_size, steps=div_steps, lamb=2)
sim.divstrat(tmax=tmax, nameDSM="./data/dataDSMsizer.csv")
print('It took', np.int(time.time()-start), 'seconds.')

start = time.time()
ncells=10000
sim = Simulator(ncells=ncells, gr=gr, sb=mean_size, steps=div_steps, lamb=0.5)
sim.divstrat(tmax=tmax, nameDSM="./data/dataDSMtimer.csv")
print('It took', np.int(time.time()-start), 'seconds.')


In [ ]:
data2 = pd.read_csv("./data/dataDSMadder.csv")
data2 = data2[data2.time > 5*doubling_time]

quantnumber = 5
pvadd2 = data2

CV2darr1 = []
deltarr1 = []
sbarr1 = []

errcv2darr1 = []
errdeltarr1 = []
errsbarr1 = []
for i in range(quantnumber):
    lperv0 = np.percentile(pvadd2.S_b, i*100/quantnumber)
    hperv0 = np.percentile(pvadd2.S_b, (i+1)*100/quantnumber)
    quanta1 = pvadd2[pvadd2.S_b > lperv0]
    quanta2 = quanta1[quanta1.S_b < hperv0]
    mean_cntr, var_cntr, std_cntr = bayesest((quanta2.S_d-quanta2.S_b)/np.mean(pvadd2.S_d-pvadd2.S_b), alpha=0.95)
    meanv0_cntr, varv0_cntr, stdv0_cntr = bayesest(quanta2.S_b/np.mean(pvadd2.S_b), alpha=0.95)
    CV2darr1.append(var_cntr[0]/mean_cntr[0]**2)
    deltarr1.append(mean_cntr[0])
    sbarr1.append(meanv0_cntr[0])
    errv = (var_cntr[1][1]-var_cntr[0])/mean_cntr[0]**2+2*(mean_cntr[1][1]-mean_cntr[0])*var_cntr[0]/mean_cntr[0]**3
    errcv2darr1.append(errv)
    errdeltarr1.append(mean_cntr[1][1]-mean_cntr[0])
    errsbarr1.append(meanv0_cntr[1][1]-meanv0_cntr[0])

data3 = pd.read_csv("./data/dataDSMsizer.csv")
data3 = data3[data3.time>5*doubling_time]

quantnumber = 5
pvadd2 = data3

CV2darr2 = []
deltarr2 = []
sbarr2 = []

errcv2darr2 = []
errdeltarr2 = []
errsbarr2 = []
for i in range(quantnumber):
    lperv0 = np.percentile(pvadd2.S_b, i*100/quantnumber)
    hperv0 = np.percentile(pvadd2.S_b, (i+1)*100/quantnumber)
    quanta1 = pvadd2[pvadd2.S_b > lperv0]
    quanta2 = quanta1[quanta1.S_b < hperv0]
    mean_cntr, var_cntr, std_cntr = bayesest((quanta2.S_d-quanta2.S_b)/np.mean(pvadd2.S_d-pvadd2.S_b), alpha=0.95)
    meanv0_cntr, varv0_cntr, stdv0_cntr = bayesest(quanta2.S_b/np.mean(pvadd2.S_b), alpha=0.95)
    CV2darr2.append(var_cntr[0]/mean_cntr[0]**2)
    deltarr2.append(mean_cntr[0])
    sbarr2.append(meanv0_cntr[0])
    errv = (var_cntr[1][1]-var_cntr[0])/mean_cntr[0]**2+2*(mean_cntr[1][1]-mean_cntr[0])*var_cntr[0]/mean_cntr[0]**3
    errcv2darr2.append(errv)
    errdeltarr2.append(mean_cntr[1][1]-mean_cntr[0])
    errsbarr2.append(meanv0_cntr[1][1]-meanv0_cntr[0])

data4 = pd.read_csv("./data/dataDSMtimer.csv")
data4 = data4[data4.time>5*doubling_time]

quantnumber = 5
pvadd2 = data4

CV2darr3 = []
deltarr3 = []
sbarr3 = []

errcv2darr3 = []
errdeltarr3 = []
errsbarr3 = []
for i in range(quantnumber):
    lperv0 = np.percentile(pvadd2.S_b,i*100/quantnumber)
    hperv0 = np.percentile(pvadd2.S_b,(i+1)*100/quantnumber)
    quanta1 = pvadd2[pvadd2.S_b>lperv0]
    quanta2 = quanta1[quanta1.S_b<hperv0]
    mean_cntr, var_cntr, std_cntr = bayesest((quanta2.S_d-quanta2.S_b)/np.mean(pvadd2.S_d-pvadd2.S_b), alpha=0.95)
    meanv0_cntr, varv0_cntr, stdv0_cntr = bayesest(quanta2.S_b/np.mean(pvadd2.S_b), alpha=0.95)
    CV2darr3.append(var_cntr[0]/mean_cntr[0]**2)
    deltarr3.append(mean_cntr[0])
    sbarr3.append(meanv0_cntr[0])
    errv = (var_cntr[1][1]-var_cntr[0])/mean_cntr[0]**2+2*(mean_cntr[1][1]-mean_cntr[0])*var_cntr[0]/mean_cntr[0]**3
    errcv2darr3.append(errv)
    errdeltarr3.append(mean_cntr[1][1]-mean_cntr[0])
    errsbarr3.append(meanv0_cntr[1][1]-meanv0_cntr[0])


In [ ]:
#Numerical estimation of the mean trends for the division strategy

sim = Simulator(ncells=1, gr=gr, sb=mean_size, steps=div_steps, lamb=0.5)
sbar = np.linspace(0.5, 1.5, 100)*mean_size
cv2tim = []
delttim = []
for i in sbar:
    sd, cv2 = sim.SdStat(i)
    cv2tim.append(cv2)
    delttim.append(sd)


sim = Simulator(ncells=1, gr=gr, sb=mean_size, steps=div_steps)
sbar = np.linspace(0.5, 1.5, 100)*mean_size
cv2ad = []
deltad = []
for i in sbar:
    sd, cv2 = sim.SdStat(i)
    cv2ad.append(cv2)
    deltad.append(sd)


sim = Simulator(ncells=1, gr = gr, sb=mean_size, steps = div_steps,lamb=2)
sbar = np.linspace(0.5,1.5,100)*mean_size
cv2sz = []
deltsz = []
for i in sbar:
    sd, cv2 = sim.SdStat(i)
    cv2sz.append(cv2)
    deltsz.append(sd)

In [ ]:
#ploting division strategy
fig, ax = plt.subplots(1, 2, figsize=(12, 4))

ax[0].errorbar(np.array(sbarr1), np.array(deltarr1), xerr=errsbarr1, yerr=errdeltarr1, fmt='o', mec='k', capsize=5,
               markersize='8', elinewidth=3, c='k')
ax[1].errorbar(np.array(sbarr1), CV2darr1, xerr=errsbarr1, yerr=errcv2darr1, fmt='o', mec='k', capsize=5,
               markersize='8', elinewidth=3, c='k')

ax[0].errorbar(np.array(sbarr2), np.array(deltarr2), xerr=errsbarr2, yerr=errdeltarr2, fmt='o', mec='k', capsize=5,
               markersize='8', elinewidth=3, c='r')
ax[1].errorbar(np.array(sbarr2), CV2darr2, xerr=errsbarr2, yerr=errcv2darr2, fmt='o', mec='k', capsize=5,
               markersize='8', elinewidth=3, c='r')

ax[0].errorbar(np.array(sbarr3), np.array(deltarr3), xerr=errsbarr3, yerr=errdeltarr3, fmt='o', mec='k', capsize=5,
               markersize='8', elinewidth=3, c='g')
ax[1].errorbar(np.array(sbarr3), CV2darr3, xerr=errsbarr3, yerr=errcv2darr3, fmt='o', mec='k', capsize=5,
               markersize='8', elinewidth=3, c='g')


ax[1].set_ylim([0, 0.3])

ax[0].set_xlabel("$s_b/\overline{s_b}$", size=20)
ax[1].set_xlabel("$s_b/\overline{s_b}$", size=20)
ax[0].set_ylabel("$\Delta/\overline{s_b}$", size=15)
ax[1].set_ylabel("$C_V^2(\Delta)$", size=15)
for l in [0, 1]:
    ax[l].grid()
    ax[l].tick_params(axis='x', labelsize=15)
    ax[l].tick_params(axis='y', labelsize=15)
    for axis in ['bottom', 'left']:
        ax[l].spines[axis].set_linewidth(2)
        ax[l].tick_params(axis='both', width=2, length=6)
    for axis in ['top', 'right']:
        ax[l].spines[axis].set_linewidth(0)
        ax[l].tick_params(axis='both', width=0, length=6)


ax[0].plot(np.array(sbar)/mean_size, np.array(delttim)/mean_size, lw=2,c='g', label="$\lambda=0.5$")
ax[1].plot(np.array(sbar)/mean_size, cv2tim, lw=2, c='g')

ax[0].plot(np.array(sbar)/mean_size, np.array(deltad)/mean_size, lw=2, c='k', label="$\lambda=1$")
ax[1].plot(np.array(sbar)/mean_size, cv2ad, lw=2, c='k')

ax[0].plot(np.array(sbar)/mean_size, np.array(deltsz)/mean_size, lw=2, c='r', label="$\lambda=2$")
ax[1].plot(np.array(sbar)/mean_size, cv2sz, lw=2, c='r')

ax[0].set_ylim(0.75,1.35)
ax[1].set_ylim(0.03,0.17)
ax[0].text(0.55, 1.27, "$\lambda = 2$", rotation=-35, fontsize=10)
ax[0].text(0.55, 1.01, "$\lambda = 1$", fontsize=10)
ax[0].text(0.55, 0.87, "$\lambda = 0.5$", rotation=35, fontsize=10)

ax[1].text(0.5, 0.05, "$\lambda = 2$", rotation=15, fontsize=10)
ax[1].text(0.5, 0.11, "$\lambda = 1$", fontsize=10)
ax[1].text(0.5, 0.155, "$\lambda = 0.5$", rotation=-10, fontsize=10)


#plt.savefig('./figures/full_div_strategy.eps',bbox_inches='tight')


In [ ]:
tmax=6
ncells=20
start = time.time()
sim = Simulator(ncells=ncells, gr=np.log(2), sb=mean_size, steps=div_steps)
sim.szdyn(tmax=tmax,sample_time=0.05,nameCRM="./data/SingleLin.csv")
print('It took', np.int(time.time()-start), 'seconds.')

In [ ]:
#ploting Figure2
fig= plt.figure(figsize=(15,2))
gs0 = gridspec.GridSpec(1, 3, figure=fig,wspace=0.2)

ax0 = fig.add_subplot(gs0[1])

data=pd.read_csv("./data/AdderDyn.csv")
ax0.plot(data.time,data.Meansize,c='#00609B')
ax0.set_ylim(0)

data=pd.read_csv("./data/dataFSPsz.csv")
ax0.plot(data.time,data.Meansize,c='r')

data=pd.read_csv("./data/dataFSPtim.csv")
ax0.plot(data.time,data.Meansize,c='g')


setaxis(ax0)

ax1 = fig.add_subplot(gs0[2])
ax1.set_ylim(0,0.3)
ax0.set_ylim(1,1.7)
ax0.set_xlim(0,6)
ax1.set_xlim(0,6)

ax0.set_title('Dynamics of the mean cell size')
ax1.set_title('Dynamics of the cell size fluctuations')

data=pd.read_csv("./data/AdderDyn.csv")
ax1.plot(data.time,data.VarSize,c='#00609B')

data=pd.read_csv("./data/dataFSPtim.csv")
ax1.plot(data.time,data.VarSize,c='g')       

data=pd.read_csv("./data/dataFSPsz.csv")
ax1.plot(data.time,data.VarSize,c='r') 
setaxis(ax1)
gs00 = gridspec.GridSpecFromSubplotSpec(1,2, subplot_spec=gs0[0],wspace=0.3)
ax20 = fig.add_subplot(gs00[0])
ax21 = fig.add_subplot(gs00[1])

ax20.set_xlim(0.45,1.55)
ax21.set_xlim(0.5,1.5)
ax20.set_ylim(0.7,1.35)
ax21.set_ylim(0,0.2)

ax20.plot(np.array(sbar)/mean_size, np.array(delttim)/mean_size, lw=2,c='g', label="$\lambda=0.5$")
ax21.plot(np.array(sbar)/mean_size, cv2tim, lw=2, c='g')

ax20.plot(np.array(sbar)/mean_size, np.array(deltad)/mean_size, lw=2, c='#00609B', label="$\lambda=1$")
ax21.plot(np.array(sbar)/mean_size, cv2ad, lw=2, c='#00609B')

ax20.plot(np.array(sbar)/mean_size, np.array(deltsz)/mean_size, lw=2, c='r', label="$\lambda=2$")
ax21.plot(np.array(sbar)/mean_size, cv2sz, lw=2, c='r')

ax20.errorbar(np.array(sbarr1), np.array(deltarr1), xerr=errsbarr1, yerr=errdeltarr1, fmt='o', mec='k',
              capsize=3,markersize='5', elinewidth=3, c='#00609B')
ax21.errorbar(np.array(sbarr1), CV2darr1, xerr=errsbarr1, yerr=errcv2darr1, fmt='o', mec='k', capsize=3,
               markersize='5', elinewidth=3, c='#00609B')

ax20.errorbar(np.array(sbarr2), np.array(deltarr2), xerr=errsbarr2, yerr=errdeltarr2, fmt='o', mec='k',
              capsize=5, markersize='5', elinewidth=3, c='r')
ax21.errorbar(np.array(sbarr2), CV2darr2, xerr=errsbarr2, yerr=errcv2darr2, fmt='o', mec='k', capsize=5,
               markersize='5', elinewidth=3, c='r')

ax20.errorbar(np.array(sbarr3), np.array(deltarr3), xerr=errsbarr3, yerr=errdeltarr3, fmt='o', mec='k',
              capsize=5, markersize='5', elinewidth=3, c='g')
ax21.errorbar(np.array(sbarr3), CV2darr3, xerr=errsbarr3, yerr=errcv2darr3, fmt='o', mec='k', 
              capsize=5, markersize='5', elinewidth=3, c='g')
setaxis(ax20)
setaxis(ax21)

ax20.set_xlabel('Size at Birth')
ax21.set_xlabel('Size at Birth')
ax20.set_ylabel('Added Size')
ax21.set_ylabel('Noise in Added Size')

ax20.set_ylabel('Division strategy')

ax0.set_xlabel('Time')
ax1.set_xlabel('Time')
ax0.set_ylabel('Cell Size')
ax1.set_ylabel('Noise in Cell Size')

ax20.text(0.65,1.21,r'Sizer-like')
ax20.text(0.45,1.02,r'Adder')
ax20.text(0.72,0.81,r'Timer-like')
plt.savefig('./figures/full_div_strategy.eps',bbox_inches='tight')

In [ ]:
#Simulation of size trajectories for coupling to gene expression
from PyEcoLib.PopSimulator import PopSimulator
sim = PopSimulator(ncells=5000,gr=np.log(2), sb=1, steps=10, nu=1,V0array=np.random.gamma(shape=1/0.03,scale=0.03,size=5000)) 

In [ ]:
sim.szdyn(tmax=5,sample_time=0.1,DivEventsFile='./data/DivEvents.csv')

In [ ]:
#implementation of gene expression algorithm
df=pd.read_csv('./data/DivEvents.csv')
dt=0.01
tmax=5

b=10
kr=10*np.log(2)
dataf=[]
for cell in df.Cell.unique():
    p=100
    dfcell=df[df.Cell==cell]
    reactimes=dfcell.BirthTime.tolist()
    grarr=dfcell.GrowthRate.tolist()
    sbarr=dfcell.Sb.tolist()
    sdarr=dfcell.MotherSize.tolist()
    td=reactimes[1]
    gr=grarr[0]
    s=sbarr[0]
    
    tr=(1/gr)*np.log(1-(gr/(s*kr))*np.log(np.random.rand()))
    
    nextt=np.min([tr,td])
    react=np.argmin([tr,td])
    t=0
    ndivs=0
    dataf.append([cell,t,p,s,p/s]) #data sampling
    while t<=tmax:
        tt=dt
        if nextt<tt:
            while nextt<tt:
                s*=np.exp(gr*nextt)
                td+=-nextt
                tr+=-nextt
                tt+=-nextt
                if react==0: #express gene
                    p+=np.random.geometric(p=1/10)
                    tr=(1/gr)*np.log(1-(gr/(s*kr))*np.log(np.random.rand()))
                else:
                    ndivs+=1
                    if ndivs<len(reactimes)-1:   
                        td=reactimes[ndivs+1]-reactimes[ndivs]
                    else:
                        td=tmax+2*dt
                    f=sbarr[ndivs]/sdarr[ndivs]
                    gr=grarr[ndivs]
                    s=sbarr[ndivs]
                    p=np.random.binomial(p,f)                    
                nextt=np.min([tr,td])
                react=np.argmin([tr,td])
            s*=np.exp(gr*tt)
            td+=-tt
            tr+=-tt
        else:
            nextt+=-dt #the sampling time is shorter than the closest reaction time
            s*=np.exp(gr*dt)
            td+=-dt
            tr+=-dt
        t+=dt
        dataf.append([cell,t,p,s,p/s]) #data sampling

    
    

In [ ]:
#Exporting gene expression data
datagen=pd.DataFrame(dataf,columns=['Cell','Time','Np','Sz','p'])
datagen.to_csv('./data/GeneExpression.csv',index=False)

In [ ]:
#ploting Figure4
datagen=pd.read_csv('./data/GeneExpression.csv')
fig, ax = plt.subplots(2, 3, figsize=(12, 4),sharex=True)

for cell in datagen.Cell.unique()[:10]:
    datacell=datagen[datagen.Cell==cell]
    ax[0,0].plot(datacell.Time,datacell.Sz,c='#008EE4',lw=0.7)

mnarr=[]
cv2arr=[]
for t in datagen.Time.unique():
    datatime=datagen[datagen.Time==t]
    mnarr.append(np.mean(datatime.Sz))
    cv2arr.append(np.var(datatime.Sz)/np.mean(datatime.Sz)**2)
ax[0,0].plot(datagen.Time.unique(),mnarr,c="#005A91")
ax[1,0].plot(datagen.Time.unique(),cv2arr,c="#005A91")

for cell in datagen.Cell.unique()[:10]:
    datacell=datagen[datagen.Cell==cell]
    ax[0,1].plot(datacell.Time,datacell.Np,c='#FE4B2B',lw=0.7)

mnarr=[]
cv2arr=[]
for t in datagen.Time.unique():
    datatime=datagen[datagen.Time==t]
    mnarr.append(np.mean(datatime.Np))
    cv2arr.append(np.var(datatime.Np)/np.mean(datatime.Np)**2)
ax[0,1].plot(datagen.Time.unique(),mnarr,c="#D32000")
ax[1,1].plot(datagen.Time.unique(),cv2arr,c="#D32000")


for cell in datagen.Cell.unique()[:10]:
    datacell=datagen[datagen.Cell==cell]
    ax[0,2].plot(datacell.Time,datacell.p,c='#AF49FF',lw=0.7)

mnarr=[]
cv2arr=[]
for t in datagen.Time.unique():
    datatime=datagen[datagen.Time==t]
    mnarr.append(np.mean(datatime.p))
    cv2arr.append(np.var(datatime.p)/np.mean(datatime.p)**2)
ax[0,2].plot(datagen.Time.unique(),mnarr,c="#7200CC")
ax[1,2].plot(datagen.Time.unique(),cv2arr,c="#7200CC")
for i in [0,1]:
    for j in [0,1,2]:
        setaxis(ax[i,j])
        ax[i,j].set_xlim(0,5)
ax[0,0].set_ylabel(r'Cell Size, $s$')
ax[0,1].set_ylabel(r'Protein Number, $n$')
ax[0,2].set_ylabel(r'Protein Concentration, $p$')

ax[1,0].set_ylabel(r'Size Fluctuations $CV^2_s$')
ax[1,1].set_ylabel(r'Number Fluctuations $CV^2_n$')
ax[1,2].set_ylabel(r'Conc. Fluctuations $CV^2_p$')

ax[1,0].set_xlabel(r'Time')
ax[1,1].set_xlabel(r'Time')
ax[1,2].set_xlabel(r'Time')

ax[1,0].set_ylim(0,0.15)
ax[1,1].set_ylim(0,0.2)
ax[1,2].set_ylim(0,0.08)

ax[0,0].set_title('Size Dynamics')
ax[0,1].set_title('Protein Number Dynamics')
ax[0,2].set_title('Protein Concentration Dynamics')

plt.savefig('./figures/gene.eps',bbox_inches='tight',dpi=600)

Estimating effects of cells division noise in gene expression

In [ ]:
#Simulating Cell trajectories and times to division
from PyEcoLib.PopSimulator import PopSimulator
for nstps in [1,2,3,5,10,20,30]:
    sim = PopSimulator(ncells=5000,gr=np.log(2), sb=1, 
                       steps=nstps, nu=1,V0array=np.random.gamma(shape=1/0.1,scale=0.1,size=5000))
    sim.szdyn(tmax=5,sample_time=0.1,DivEventsFile=f'./data/DivEvents{nstps}.csv')

In [ ]:
#Changing number of division steps
dataf=[]
for nstps in [1,2,3,5,10,20,30]:

    df=pd.read_csv(f'./data/DivEvents{nstps}.csv')
    dt=0.01
    tmax=5

    kr=4*np.log(2)
    
    for cell in tqdm(df.Cell.unique()):
        p=np.random.poisson(lam=10)
        dfcell=df[df.Cell==cell]
        reactimes=dfcell.BirthTime.tolist()
        grarr=dfcell.GrowthRate.tolist()
        sbarr=dfcell.Sb.tolist()
        sdarr=dfcell.MotherSize.tolist()
        if len(reactimes)>1:
            td=reactimes[1]
            gr=grarr[0]
            s=sbarr[0]

            tr=(1/gr)*np.log(1-(gr/(s*kr))*np.log(np.random.rand()))

            nextt=np.min([tr,td])
            react=np.argmin([tr,td])
            t=0
            ndivs=0
            dataf.append([nstps,cell,t,p,s,p/s]) #data sampling
            while t<=tmax:
                tt=dt
                if nextt<tt:
                    while nextt<tt:
                        s*=np.exp(gr*nextt)
                        td+=-nextt
                        tr+=-nextt
                        tt+=-nextt
                        if react==0: #express gene
                            p+=np.random.geometric(p=1/5)
                            tr=(1/gr)*np.log(1-(gr/(s*kr))*np.log(np.random.rand()))
                        else:
                            ndivs+=1
                            if ndivs<len(reactimes)-1:   
                                td=reactimes[ndivs+1]-reactimes[ndivs]
                            else:
                                td=tmax+2*dt
                            f=sbarr[ndivs]/sdarr[ndivs]
                            gr=grarr[ndivs]
                            s=sbarr[ndivs]
                            p=np.random.binomial(p,f)                    
                        nextt=np.min([tr,td])
                        react=np.argmin([tr,td])
                    s*=np.exp(gr*tt)
                    td+=-tt
                    tr+=-tt
                else:
                    nextt+=-dt #the sampling time is shorter than the closest reaction time
                    s*=np.exp(gr*dt)
                    td+=-dt
                    tr+=-dt
                t+=dt
                dataf.append([nstps,cell,t,p,s,p/s]) #data sampling


    

In [ ]:
#Saving different division steps
datagen=pd.DataFrame(dataf,columns=['Stp','Cell','Time','Np','Sz','p'])
datagen.to_csv('./data/GeneExpressionstp.csv',index=False)

In [ ]:
#Estimating the statistics for differnet division steps
data=pd.read_csv('./data/GeneExpressionstp.csv')
for stp in data.Stp.unique():
    datastp=data[data.Stp==stp]
    tmax=datastp.Time.tolist()[-1]
    datat=datastp[datastp.Time==tmax]
    print(np.mean(datat.p),np.var(datat.p)/np.mean(datat.p))

In [ ]:
#Simulating cell size trajectories for different noise levels in growth rate
from PyEcoLib.PopSimulator import PopSimulator
ncells=5000
for cv2gr in tqdm([0.001,0.01,0.02,0.04,0.06,0.08,0.1]):
    sim = PopSimulator(ncells=ncells,gr=np.log(2), sb=1, 
                       steps=10, nu=1,V0array=np.random.gamma(shape=1/0.5,scale=0.5,size=ncells),CV2gr=cv2gr)
    sim.szdyn(tmax=5,sample_time=0.1,DivEventsFile=f'./data/DivEventscv2gr{int(10000*cv2gr)}.csv')

In [ ]:
#Simulating Gene expression for different noise levels in growth rate
dataf=[]
for cv2gr in [0.001,0.01,0.02,0.04,0.06,0.08,0.1]:

    df=pd.read_csv(f'./data/DivEventscv2gr{int(10000*cv2gr)}.csv')
    dt=0.01
    tmax=5

    b=1
    kr=4*np.log(2)
    
    for cell in tqdm(df.Cell.unique()):
        p=np.random.poisson(lam=10)
        dfcell=df[df.Cell==cell]
        reactimes=dfcell.BirthTime.tolist()
        grarr=dfcell.GrowthRate.tolist()
        sbarr=dfcell.Sb.tolist()
        sdarr=dfcell.MotherSize.tolist()
        if len(reactimes)>1:
            td=reactimes[1]
            gr=grarr[0]
            s=sbarr[0]

            tr=(1/gr)*np.log(1-(gr/(s*kr))*np.log(np.random.rand()))

            nextt=np.min([tr,td])
            react=np.argmin([tr,td])
            t=0
            ndivs=0
            dataf.append([cv2gr,cell,t,p,s,p/s]) #data sampling
            while t<=tmax:
                tt=dt
                if nextt<tt:
                    while nextt<tt:
                        s*=np.exp(gr*nextt)
                        td+=-nextt
                        tr+=-nextt
                        tt+=-nextt
                        if react==0: #express gene
                            p+=np.random.geometric(p=1/5)
                            tr=(1/gr)*np.log(1-(gr/(s*kr))*np.log(np.random.rand()))
                        else:
                            ndivs+=1
                            if ndivs<len(reactimes)-1:   
                                td=reactimes[ndivs+1]-reactimes[ndivs]
                            else:
                                td=tmax+2*dt
                            f=sbarr[ndivs]/sdarr[ndivs]
                            gr=grarr[ndivs]
                            s=sbarr[ndivs]
                            p=np.random.binomial(p,f)                    
                        nextt=np.min([tr,td])
                        react=np.argmin([tr,td])
                    s*=np.exp(gr*tt)
                    td+=-tt
                    tr+=-tt
                else:
                    nextt+=-dt #the sampling time is shorter than the closest reaction time
                    s*=np.exp(gr*dt)
                    td+=-dt
                    tr+=-dt
                t+=dt
                dataf.append([cv2gr,cell,t,p,s,p/s]) #data sampling

In [ ]:
#Saving data
datagen=pd.DataFrame(dataf,columns=['CV2gr','Cell','Time','Np','Sz','p'])
datagen.to_csv('./data/GeneExpressioncv2gr.csv',index=False)

In [ ]:
#Simulating cell size trajectories for different noise levels in partitioning position
from PyEcoLib.PopSimulator import PopSimulator
ncells=5000
for cv2div in tqdm([0.001,0.01,0.02,0.04,0.06,0.08,0.1]):
    sim = PopSimulator(ncells=ncells,gr=np.log(2), sb=1, 
                       steps=10, nu=1,V0array=np.random.gamma(shape=1/0.5,scale=0.5,size=ncells),
                       CV2div=cv2div)
    sim.szdyn(tmax=5,sample_time=0.1,DivEventsFile=f'./data/DivEventscv2div{int(10000*cv2div)}.csv')

In [ ]:
#Simulating gene expression for different noise levels in partitioning position
dataf=[]

for cv2div in [0.001,0.01,0.02,0.04,0.06,0.08,0.1]:
    df=pd.read_csv(f'./data/DivEventscv2div{int(10000*cv2div)}.csv')
    dt=0.01
    tmax=5

    b=1
    kr=4*np.log(2)
    
    for cell in tqdm(df.Cell.unique()):
        p=np.random.poisson(lam=10)
        dfcell=df[df.Cell==cell]
        reactimes=dfcell.BirthTime.tolist()
        grarr=dfcell.GrowthRate.tolist()
        sbarr=dfcell.Sb.tolist()
        sdarr=dfcell.MotherSize.tolist()
        if len(reactimes)>1:
            td=reactimes[1]
            gr=grarr[0]
            s=sbarr[0]

            tr=(1/gr)*np.log(1-(gr/(s*kr))*np.log(np.random.rand()))

            nextt=np.min([tr,td])
            react=np.argmin([tr,td])
            t=0
            ndivs=0
            dataf.append([cv2div,cell,t,p,s,p/s]) #data sampling
            while t<=tmax:
                tt=dt
                if nextt<tt:
                    while nextt<tt:
                        s*=np.exp(gr*nextt)
                        td+=-nextt
                        tr+=-nextt
                        tt+=-nextt
                        if react==0: #express gene
                            p+=np.random.geometric(p=1/5)
                            tr=(1/gr)*np.log(1-(gr/(s*kr))*np.log(np.random.rand()))
                        else:
                            ndivs+=1
                            if ndivs<len(reactimes)-1:   
                                td=reactimes[ndivs+1]-reactimes[ndivs]
                            else:
                                td=tmax+2*dt
                            f=sbarr[ndivs]/sdarr[ndivs]
                            gr=grarr[ndivs]
                            s=sbarr[ndivs]
                            p=np.random.binomial(p,f)                    
                        nextt=np.min([tr,td])
                        react=np.argmin([tr,td])
                    s*=np.exp(gr*tt)
                    td+=-tt
                    tr+=-tt
                else:
                    nextt+=-dt #the sampling time is shorter than the closest reaction time
                    s*=np.exp(gr*dt)
                    td+=-dt
                    tr+=-dt
                t+=dt
                dataf.append([cv2div,cell,t,p,s,p/s]) #data sampling


    

In [ ]:
#Saving data
datagen=pd.DataFrame(dataf,columns=['CV2div','Cell','Time','Np','Sz','p'])
datagen.to_csv('./data/GeneExpressioncv2div.csv',index=False)

In [ ]:
#Estimating statistics 
datagen=pd.read_csv('./data/GeneExpressioncv2gr.csv')
for stp in datagen.CV2gr.unique():
    datastp=datagen[datagen.CV2gr==stp]
    tmax=datastp.Time.tolist()[-1]
    datat=datastp[datastp.Time==tmax]
    print(np.mean(datat.p),np.var(datat.p)/np.mean(datat.p)**2)

In [ ]:
#Estimating statistics
datagen=pd.read_csv('./data/GeneExpressioncv2div.csv')
for stp in datagen.CV2div.unique():
    datastp=datagen[datagen.CV2div==stp]
    tmax=datastp.Time.tolist()[-1]
    datat=datastp[datastp.Time==tmax]
    print(np.mean(datat.p),np.var(datat.p)/np.mean(datat.p)**2)

In [ ]:
import random
def bootstrap(arr):
    mnar=np.empty(100)
    cv2ar=np.empty(100)
    for l in range(100):
        boot=random.choices(arr,k=len(arr))
        mn=np.mean(boot)
        cv2=np.var(boot)/np.mean(boot)**2
        mnar[l]=mn
        cv2ar[l]=cv2
    mn=np.mean(mnar)
    up=np.quantile(mnar,0.95)
    down=np.quantile(mnar,0.05)
    cv2=np.mean(cv2ar)
    up2=np.quantile(cv2ar,0.95)
    down2=np.quantile(cv2ar,0.05)
    return(mn,0.5*(-down+up),cv2,0.5*(-down2+up2))

In [ ]:
#Ploting Figure5
props2 = dict(boxstyle='round',facecolor= '#FFFFFF',linewidth=1,edgecolor='#AEAEAE')

fig= plt.figure(figsize=(13,5))
gs0 = gridspec.GridSpec(2, 3, figure=fig,wspace=0.25,hspace=0.4)

gs00 = gridspec.GridSpecFromSubplotSpec(2,2, subplot_spec=gs0[0,0],wspace=0.1)
ax20 = fig.add_subplot(gs00[0,0])
ax21 = fig.add_subplot(gs00[1,0])





data=pd.read_csv('./data/GeneExpressionstp.csv')
df=data[data.Stp==10]
cmap = matplotlib.cm.get_cmap('YlGn')
color=np.linspace(0.2,0.8,10)

i=0
for cell in df.Cell.unique()[0:10]:
    dfcell=df[df.Cell==cell]
    clr=cmap(color[i])
    ax20.plot(dfcell.Time,dfcell.Sz,c=clr,lw=1)
    ax21.plot(dfcell.Time,dfcell.p,c=clr,lw=1)
    i+=1


ax20.set_ylabel('Cell Size')
ax20.text(0.3,4.5,r'steps=10',bbox=props2,fontsize=9)
ax21.text(0.3,65,r'steps=10',bbox=props2,fontsize=9)
ax20.set_ylim(0,6)
ax21.set_ylim(0,90)
ax21.set_ylabel('Prot. Conc.')
ax20.set_xlim(0,5)
ax21.set_xlim(0,5) 
setaxis(ax21)
setaxis(ax20)
ax21.set_xlabel('time')
    
ax20 = fig.add_subplot(gs00[0,1])
ax21 = fig.add_subplot(gs00[1,1])

data=pd.read_csv('./data/GeneExpressionstp.csv')
df=data[data.Stp==1]
i=0
for cell in df.Cell.unique()[20:30]:
    dfcell=df[df.Cell==cell]
    clr=cmap(color[i])
    ax20.plot(dfcell.Time,dfcell.Sz,c=clr,lw=1)
    ax21.plot(dfcell.Time,dfcell.p,c=clr,lw=1)
    i+=1

ax20.text(0.3,4.5,r'steps=1',bbox=props2,fontsize=9)
ax21.text(0.3,65,r'steps=1',bbox=props2,fontsize=9)
ax20.set_ylim(0,6)
ax21.set_ylim(0,90)
ax20.set_xlim(0,5)
ax21.set_xlim(0,5) 
setaxis(ax21)
setaxis(ax20)
ax21.set_xlabel('time')

ax10=fig.add_subplot(gs0[1,0])


noisesar=[]
noiseser=[]
noisesz=[]
noisesz=[]
for stp in data.Stp.unique():
    datastp=data[data.Stp==stp]
    tmax=datastp.Time.tolist()[-1]
    datat=datastp[datastp.Time==tmax]
    parr=datat.p.tolist()
    mn,err,cv2,cv2err=bootstrap(parr)
    noisesar.append(cv2)
    noiseser.append(cv2err)
    noisesz.append(1/stp)

ax10.errorbar(noisesz,noisesar,yerr=noiseser,capsize=3,fmt='o',ls='--',color=clr,markersize=5)
setaxis(ax10)
ax10.set_ylim(0.15,0.25)
ax10.set_yticks([0.15,0.20,0.25])
ax10.set_ylabel(r'Protein Noise, $CV^2_p$')
ax10.set_xlabel(r'Noise in Added Size, $CV^2_\Delta$')
ax10.text(0.05,0.23,r'$CV^2_{gr}=0,CV^2_{div}=0$',bbox=props2)
#ax10.set_ylim(0)
#==============================================================================

gs00 = gridspec.GridSpecFromSubplotSpec(2,2, subplot_spec=gs0[0,1],wspace=0.1)
ax20 = fig.add_subplot(gs00[0,0])
ax21 = fig.add_subplot(gs00[1,0])

data=pd.read_csv('./data/GeneExpressioncv2gr.csv')
df=data[data.CV2gr==0.001]
i=0
cmap = matplotlib.cm.get_cmap('BuPu')
for cell in df.Cell.unique()[0:10]:
    dfcell=df[df.Cell==cell]
    clr=cmap(color[i])
    ax20.plot(dfcell.Time,dfcell.Sz,c=clr,lw=1)
    ax21.plot(dfcell.Time,dfcell.p,c=clr,lw=1)
    i+=1
ax20.text(0.3,4.5,r'$CV^2_{gr}=10^{-3}$',bbox=props2,fontsize=9)
ax21.text(0.3,65,r'$CV^2_{gr}=10^{-3}$',bbox=props2,fontsize=9)
ax20.set_ylim(0,6)
ax21.set_ylim(0,90)
ax20.set_ylabel('Cell Size')

ax21.set_ylabel('Prot. Conc.')
ax20.set_xlim(0,5)
ax21.set_xlim(0,5) 
setaxis(ax21)
setaxis(ax20)
ax21.set_xlabel('time')

ax20 = fig.add_subplot(gs00[0,1])
ax21 = fig.add_subplot(gs00[1,1])

df=data[data.CV2gr==0.1]
i=0
for cell in df.Cell.unique()[0:10]:
    dfcell=df[df.Cell==cell]
    clr=cmap(color[i])
    ax20.plot(dfcell.Time,dfcell.Sz,c=clr,lw=1)
    ax21.plot(dfcell.Time,dfcell.p,c=clr,lw=1)
    i+=1
ax20.text(0.3,4.5,r'$CV^2_{gr}=10^{-1}$',bbox=props2,fontsize=9)
ax21.text(0.3,65,r'$CV^2_{gr}=10^{-1}$',bbox=props2,fontsize=9)
ax20.set_ylim(0,6)
ax21.set_ylim(0,90)
ax20.set_xlim(0,5)
ax21.set_xlim(0,5) 
setaxis(ax21)
setaxis(ax20)
ax21.set_xlabel('time')

ax10=fig.add_subplot(gs0[1,1])

noisesar=[]
noiseser=[]
noisesz=[]
noisesz=[]
for stp in data.CV2gr.unique():
    datastp=data[data.CV2gr==stp]
    tmax=datastp.Time.tolist()[-1]
    datat=datastp[datastp.Time==tmax]
    parr=datat.p.tolist()
    mn,err,cv2,cv2err=bootstrap(parr)
    noisesar.append(cv2)
    noiseser.append(cv2err)
    noisesz.append(stp)

ax10.errorbar(noisesz,noisesar,yerr=noiseser,capsize=3,fmt='o',ls='--',color=clr,markersize=5)
setaxis(ax10)
ax10.set_ylim(0.15,0.25)
ax10.set_yticks([0.15,0.20,0.25])
ax10.set_ylabel(r'Protein Noise, $CV^2_p$')
ax10.set_xlabel(r'Noise in Growth rate, $CV^2_{gr}$')
ax10.text(0.001,0.23,r'steps$=10,CV^2_{div}=0$',bbox=props2)
#ax10.set_ylim(0)
#================================================================

gs00 = gridspec.GridSpecFromSubplotSpec(2,2, subplot_spec=gs0[0,2],wspace=0.1)
ax20 = fig.add_subplot(gs00[0,0])
ax21 = fig.add_subplot(gs00[1,0])

data=pd.read_csv('./data/GeneExpressioncv2div.csv')
df=data[data.CV2div==0.001]

cmap = matplotlib.cm.get_cmap('GnBu')
i=0

for cell in df.Cell.unique()[10:20]:
    dfcell=df[df.Cell==cell]
    clr=cmap(color[i])
    ax20.plot(dfcell.Time,dfcell.Sz,c=clr,lw=1)
    ax21.plot(dfcell.Time,dfcell.p,c=clr,lw=1)
    i+=1
ax20.text(0.3,4.5,r'$CV^2_{div}=10^{-3}$',bbox=props2,fontsize=9)
ax21.text(0.3,65,r'$CV^2_{div}=10^{-3}$',bbox=props2,fontsize=9)
ax20.set_ylim(0,6)
ax21.set_ylim(0,90)
ax20.set_ylabel('Cell Size')
ax21.set_ylabel('Prot. Conc.')
ax20.set_xlim(0,5)
ax21.set_xlim(0,5) 
setaxis(ax21)
setaxis(ax20)
ax21.set_xlabel('time')

ax20 = fig.add_subplot(gs00[0,1])
ax21 = fig.add_subplot(gs00[1,1])

df=data[data.CV2div==0.1]
i=0
for cell in df.Cell.unique()[0:10]:
    dfcell=df[df.Cell==cell]
    clr=cmap(color[i])
    ax20.plot(dfcell.Time,dfcell.Sz,c=clr,lw=1)
    ax21.plot(dfcell.Time,dfcell.p,c=clr,lw=1)
    i+=1
ax20.text(0.3,4.5,r'$CV^2_{div}=10^{-1}$',bbox=props2,fontsize=9)
ax21.text(0.3,65,r'$CV^2_{div}=10^{-1}$',bbox=props2,fontsize=9)
ax20.set_ylim(0,6)
ax21.set_ylim(0,90)
ax20.set_xlim(0,5)
ax21.set_xlim(0,5)
setaxis(ax21)
setaxis(ax20)  
ax21.set_xlabel('time')

ax10=fig.add_subplot(gs0[1,2])

noisesar=[]
noiseser=[]
noisesz=[]
noisesz=[]
for stp in data.CV2div.unique():
    datastp=data[data.CV2div==stp]
    tmax=datastp.Time.tolist()[-1]
    datat=datastp[datastp.Time==tmax]
    parr=datat.p.tolist()
    mn,err,cv2,cv2err=bootstrap(parr)
    noisesar.append(cv2)
    noiseser.append(cv2err)
    noisesz.append(stp)

ax10.errorbar(noisesz,noisesar,yerr=noiseser,capsize=3,fmt='o',ls='--',color=clr,markersize=5)
setaxis(ax10)
#,
#                   markersize='3',elinewidth=1,fmt='o--')
ax10.set_ylim(0.15,0.25)
ax10.set_yticks([0.15,0.20,0.25])
ax10.set_ylabel(r'Protein Noise, $CV^2_p$')
ax10.set_xlabel(r'Noise in Septum Position, $CV^2_{div}$')

ax10.text(0.001,0.23,r'steps$=10,CV^2_{gr}=0$',bbox=props2)
plt.savefig('./figures/noisesources.eps',bbox_inches='tight',dpi=400)
